# Exercise 1.04: Tic-Tac-Toe Static Evaluation with a Heuristic Function
In this exercise, you will be performing a static evaluation on the tic-tac-toe game using a heuristic function.

1.- Reuse the code from Steps 1–5 of Activity 1.01

In [1]:
from random import choice
combo_indices = [[1,2,3],[4,5,6],[7,8,9],[1,4,7],[2,5,8],[3,6,9],[1,5,9],[3,5,7]]

emptyC = '#'
AI = 'X'
OP = 'O'

def Tablero(tabla):
    print(" ")
    print(' '.join(tabla[0:3:1]))
    print(' '.join(tabla[3:6:1]))
    print(' '.join(tabla[6:9:1]))
    print(" ")

def movimientoPersona(tabla, fila, columna):
    movimientoP = (3 * (fila - 1)) + (columna - 1) 
    if (tabla[movimientoP] == emptyC):
        tabla = tabla[0:movimientoP] + OP + tabla[movimientoP+1:9] 
    return tabla

def all_moves_from_board(board, sign):
    contador = 0
    opciones = []
    posiciones = []
    for v in board:
        if (v == emptyC): 
            posiciones.append(contador)
        contador+=1
    for a in posiciones:
        opciones.append(board[0:a] + sign + board[a+1:9])
    return (opciones)

def ai_move(board):
    movimientoAI = choice(all_moves_from_board(board,AI))       
    return movimientoAI 

def game_won_by(tablero):
    for item in combo_indices:
        if (tablero[item[0]-1] == tablero[item[1]-1] == tablero[item[2]-1] == OP):
            return OP
        elif (tablero[item[0]-1] == tablero[item[1]-1] == tablero[item[2]-1] == AI):
            return AI
    return emptyC

def game_loop():
    board = emptyC * 9
    empty_cell_count = 9
    winner = "#"
    while (empty_cell_count >= 0 and  (winner !="#")): 
        print(empty_cell_count)
        if (empty_cell_count % 2 == 0):
            board = ai_move(board)
            Tablero(board)
        else:
            fila = int(input('Elija la fila.'))
            columna = int(input('Elija la columna'))
            board = movimientoPersona(board,fila,columna)    
        winner = game_won_by(board)
        if(winner != emptyC):
            Tablero(board)
            break     
        empty_cell_count-=1
    print("El juego termino")
    
def Partidas(tablero, signo):
    jugadas = []
    for board in tablero:
        jugadas.extend(all_moves_from_board(board, signo))
    return jugadas

def filter_wins(tableros, AI_wins, OP_wins):
    for tab in tableros:
        win = game_won_by(tab)
        if (win == AI):
            AI_wins.append(tab)
            tableros.remove(tab)
        elif (win  == OP):
            OP_wins.append(tab)
            tableros.remove(tab)
            
def count_possibilities():
    jugadas = [emptyC * 9]
    AI_wins = []
    OP_wins = []
    print ("Possible Sequences of Steps:")
    for i in range(9):
        print('step ' + str(i) + '. Moves: ' + str(len(jugadas)))
        if (i % 2 == 0):
            signo = AI
        elif (i % 2 == 1):
            signo = OP
        jugadas = Partidas(jugadas, signo)
        filter_wins(jugadas, AI_wins, OP_wins)
    print('First player wins: ' + str(len(AI_wins)))
    print('Second player wins: ' + str(len(OP_wins)))
    print('Draw', str(len(jugadas)))
    print('Total', str(len(AI_wins) + len(OP_wins) + len(jugadas)))

2.- Create a function that takes the board as input and returns $0$ if the cell is empty, and $-1$ if it's not empty

In [2]:
"""
La función de este metodo es la siguiente compara cada una de las celdas del tablero, si una celda del tablero es igual a emptyC se añadira 
un 0 a la lista "lista", en caso contrario se añadira un -1 a la lista "lista".
Despues de terminar de comparar y añadir 0 y -1 a la lista, esta sera retornada."""
def comparador(tablero):
    lista = []
    for celda in tablero:
        if celda == emptyC:
            lista.append(0)
        else:
            lista.append(-1)
    return lista

3.- Create a function that takes the utility vector of possible moves, takes three indices inside the utility vector representing a triple, and returns a function.

  > **Hints**  
  > the returned function will expect a `points` parameter and the `utilities` vector as input and will add points to each cell in $(i, j, k)$, as long as the original value of that cell is non-negative $(>=0)$. In other words, we increased the utility of empty cells only.

In [3]:
"""
Este metodo recibe la lista que fue retornada del metodo anterior y tambien recibe los indices que se tienen en la variable "combo_indices"
y dentro de del metodo se encuentra un metodo interno que va sumando los puntos recibidos del cumplimiento de las reglas que se han
establecido"""
def generate_add_score(utilities,i,j,k):
    def evaluar(points):
        if utilities[i-1] >= 0:
            utilities[i-1] += points
        if utilities[j-1] >= 0:
            utilities[j-1] += points
        if utilities[k-1] >= 0:
            utilities[k-1] += points
    return evaluar

4.- Now, create the utility matrix belonging to any board constellation where you will add the `generate_add_score` function defined previously to update the score. You will also implement the rules that we discussed prior to this activity. These rules are as follows:
  * Two AI signs in a row, column, or diagonal, and the third cell is empty: +1000 for the empty cell.
  * The opponent has two signs in a row, column, or diagonal, and the third cell is empty: +100 for the empty cell.
  * One AI sign in a row, column, or diagonal, and the other two cells are empty: +10 for the empty cells.
  * No AI or opponent signs in a row, column, or diagonal: +1 for the empty cells.

In [4]:
"""
Lo que refleja este metodo es el cumplimiento de las reglas establecidas en el punto 4 para darle una puntuacion a cada una de las celdas
dentro de la matriz de utilidades, la variable "puntuacion" toma la lista que es retornada del metodo "comparador", despues se hace un
for para sacar los 3 valores de los indices de la variable "combo_indices", esto se hace para tener los indices que son usados para poder
ganar en el juego del gato.
los indices son guardados en las variables "i", "j", "k", ya dentro del for a la variable "evaluar" se le es asigando el metodo
generate_add_score, pero ya habiendo visto el metodo ya mencionado y sabiendo que este cuenta el retorno de su metodo interno, se da a 
entender que la variable "evaluar" de alguna manera se convierte en el metodo interno de generate_add_score.
la variable "indices", es una lista en la cual se van guardando los valores de los indices "i","j","k" dentro del tablero,
cada uno de los if que se pueden ver, son el cumplimiento de cada una de las reglas dichas en el punto 4, si alguna de estas se cumple a la
variable "eavluar", que ahora se comporta como el metodo interno de generate_add_score, se le pasa la puntuacion que se merece al cumplir
con alguna de las reglas propuestas anteriormente.
Al terminar de evaluar la variable puntuación es retornada"""
def utility_matrix(tablero):
    puntuacion = comparador(tablero)
    for [i,j,k] in combo_indices:
        evaluar = generate_add_score(puntuacion,i,j,k)
        indices = [tablero[i-1],tablero[j-1],tablero[k-1]]
        if indices.count(AI) == 2 and indices.count(emptyC) == 1:
            evaluar(1000)
        if indices.count(OP) == 2 and indices.count(emptyC) == 1:
            evaluar(100)
        if indices.count(AI) == 1 and indices.count(emptyC) == 2:
            evaluar(10)
        if indices.count(emptyC) == 3:
            evaluar(1)
    return puntuacion

5.- Create a function that selects the move with the highest utility value. If multiple moves have the same utility, the function returns both moves

In [5]:
"""
EL metodo de mejor_movimiento recibe el tablero y el signo de la IA, lo principalmente que sea hace aqui es la seleccion del mejor movimiento
que la IA puede hacer para poder ganar, la variable "puntuacion" recibie la matriz de utilidades y la variable "mejor_puntuacion" almacena
la mejor puntuacion que se encuentra dentro de la matriz de utilidades, en el metodo for lo que se va haciendo es la busqueda de la mejor
decisión que puede tomar la IA para poder ganar, cuando la variable "puntuacion" en la posicion de la variable "contador" es igual a la
variable "mejor_puntuacion", el tablero sera añadaido a la lista "movimientos".
Despues de la comparación dentro del metodo for, la lista "movimientos" sera retornada.
"""
def mejor_movimiento(tablero, signo):
    movimientos = []
    contador = 0
    puntuacion = utility_matrix(tablero)
    mejor_puntuacion = max(puntuacion)
    for v in tablero:
        if puntuacion[contador] == mejor_puntuacion:
            movimientos.append(tablero[:contador] + signo + tablero[contador+1:])
        contador+=1
    return movimientos

"""
Este metodo lo que hace es ir agrupando las jugadas que tanto IA como el oponente hacen, este metodo recible el tablero y el signo del jugador
que esta jugando en ese momento determinado, si es el IA, la variable "movimientos" funcionara como el metodo "mejor_movimiento" y si es el
oponente funcionara como el metodo "all_moves_from_board", dentro del for se hara uno de los dos metodos dependiendo de que jugador es y al
final este retornara la lista de jugadas que se pueden hacer para cada jugador
"""
def Partidas(tablero, signo):
    jugadas = []
    if signo == AI:
        movimientos = mejor_movimiento
    else:
        movimientos = all_moves_from_board
    for board in tablero:
        jugadas.extend(movimientos(board, signo))
    return jugadas

6.- Run the application.

Output:

```
step 0. Moves: 1
step 1. Moves: 1
step 2. Moves: 8
step 3. Moves: 24
step 4. Moves: 144
step 5. Moves: 83
step 6. Moves: 214
step 7. Moves: 148
step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607
```

In [6]:
"""El metodo count_possiblities es llamado"""
count_possibilities()

Possible Sequences of Steps:
step 0. Moves: 1
step 1. Moves: 1
step 2. Moves: 8
step 3. Moves: 24
step 4. Moves: 144
step 5. Moves: 83
step 6. Moves: 214
step 7. Moves: 148
step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607


By completing this exercise, we have observed that the AI is underperforming compared to our previous activity, Activity 1.03, Fixing the First and Second Moves of the AI to Make It Invincible. In this situation, hardcoding the first two moves was better than setting up the heuristic, but this is because we haven't set up the heuristic properly.